# Coleta e tratamento de dados

In [1]:
import requests
import pandas as pd

def buscar(url):
    """
    Busca dados do SIDRA sobre empresas de construção por território
    
    """    
    response = requests.get(url)
    if response.status_code == 200:        
        # Se a solicitação for bem-sucedida, formata e retorna os dados
        dados = response.json()
        df = pd.DataFrame(dados)
        df = df[['D1N','D2N','V']]
        df.columns = ['ano', 'territorio', 'num_empresas']
        return df
    else:
        #Se não retona mensagem de erro
        raise Exception("Erro ao buscar dados do SIDRA")

In [2]:
dados_regioes = buscar("https://apisidra.ibge.gov.br/values/t/1757/p/2007-2020/n2/all/v/410/h/n")

In [3]:
caminho_pop = '../data/raw/pop.xlsx'
regioes = ['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste']
idades = list(range(38, 59))
anos = list(range(2007, 2021))

def processar_dados_populacao(caminho_arquivo_pop, df_empresas, regioes, idades, anos):
    """
    Processa os dados de população e empresas, calculando a razão população/empresas.
    
    Parâmetros:
        caminho_arquivo_pop (str): Caminho do arquivo Excel com dados de população.
        df_empresas (pd.DataFrame): DataFrame com dados de empresas.
        regioes (list): Lista de regiões de interesse.
        idades (list): Lista de idades de interesse.
        anos (list): Lista de anos de interesse.
    
    Retorna:
        pd.DataFrame: DataFrame processado com a razão população/empresas.
    """
    # Ler os dados da população
    dados_pop = pd.read_excel(caminho_arquivo_pop, skiprows=5)
    
    # Filtrar os dados
    dados_filtrados = dados_pop[(dados_pop['LOCAL'].isin(regioes)) & (dados_pop['IDADE'].isin(idades))]
    
    # Selecionar colunas de interesse
    dados_filtrados = dados_filtrados[['IDADE', 'LOCAL'] + anos]
    
    # Transformar de formato largo para longo
    df_pop = dados_filtrados.melt(id_vars=["IDADE", "LOCAL"], var_name="ano", value_name="populacao")
    
    # Converter colunas para o tipo adequado
    df_pop["ano"] = df_pop["ano"].astype(int)
    
    # Agrupar por ano e região, somando a população filtrada
    df_pop = df_pop.groupby(["ano", "LOCAL"])["populacao"].sum().reset_index()
    
    # Renomear a coluna LOCAL para territorio
    df_pop.rename(columns={"LOCAL": "territorio"}, inplace=True)
    
    # Converter colunas para o tipo adequado
    df_empresas["ano"] = df_empresas["ano"].astype(int)
    df_empresas["num_empresas"] = df_empresas["num_empresas"].astype(int)
    
    # Unir os DataFrames pela coluna "ano" e "territorio"
    df_final = pd.merge(df_empresas, df_pop, on=["ano", "territorio"], how="inner")
    
    # Criar a razão da população pela quantidade de empresas
    df_final["razao_pop_empresa"] = df_final["populacao"] / df_final["num_empresas"]
    
    return df_final

In [4]:
df_resultado = processar_dados_populacao(caminho_pop, dados_regioes, regioes, idades, anos)

In [5]:
df_resultado.to_csv('../data/processed/pronto.csv', index=False)